In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 16.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pulp as pl
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [3]:
# Which solvers are available to us?
solver_list = pl.listSolvers(onlyAvailable=True)
print(solver_list)

['PULP_CBC_CMD']


In [4]:
# However Pulp can interact with multiple solvers (if installed)
solver_list = pl.listSolvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY', 'HiGHS', 'HiGHS_CMD', 'COPT', 'COPT_DLL', 'COPT_CMD']


#Resource Allocation Problem

* Two products: Product 1 generates a profit of 80, while Product 2 yields a profit of 65
* Producing Product 1 requires 0.15 hours of cutting, 0.6 hours of sowing and 0.05 of packaging
* Product Product 2 requires 0.1 hours of cutting, 0.9 hours of sowing and 0.05 hours of packaging
* The following resource contraints applies: 110 hours of cutting, 650 hours of sowing and 40 hours of packaging

Find the best product mix within resource limitations to maximize profit

Source: https://science-gym.dk/mat/20002010/lin-prog.pdf




### Finding the optimal solution to the problem

In [10]:
model = LpProblem(name="resournce-allocation", sense=LpMaximize) # Maximise profit

# Initialize the decision variables
x1 = LpVariable(name="x1", lowBound=0, upBound=None, cat='Integer') # x1 => 0
x2 = LpVariable(name="x2", lowBound=0, upBound=None, cat='Integer')

# Add the constraints to the model
model += (0.15 * x1 + 0.1  * x2 <= 110, "cutting (manpower)")
model += ( 0.6 * x1 + 0.9  * x2 <= 650, "sowing (manpower)")
model += (0.05 * x1 + 0.05 * x2 <= 40,  "packaging (manpower)")


# Add the objective function to the model
model += 80 * x1 + 65 * x2

print(model)

# Solve the problem
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}\n")

# Maximum profit
print(f"objective: {model.objective.value()}\n")

# Optimal values
for var in model.variables():
  print(f"{var.name}: {var.value()}")

resournce-allocation:
MAXIMIZE
80*x1 + 65*x2 + 0
SUBJECT TO
cutting_(manpower): 0.15 x1 + 0.1 x2 <= 110

sowing_(manpower): 0.6 x1 + 0.9 x2 <= 650

packaging_(manpower): 0.05 x1 + 0.05 x2 <= 40

VARIABLES
0 <= x1 Integer
0 <= x2 Integer

status: 1, Optimal

objective: 61000.0

x1: 600.0
x2: 200.0


### Sensitivity Analysis

In [6]:
# Display Contraints - Shahow price and Slack
o = [{
    'name':name,
    'shadow price':c.pi,
    'slack': c.slack} for name, c in model.constraints.items()]

print(pd.DataFrame(o))

                   name  shadow price  slack
0    cutting_(manpower)         300.0   -0.0
1     sowing_(manpower)          -0.0  110.0
2  packaging_(manpower)         700.0   -0.0


**Observation**

 Cutting and Packaging are binding (slack = 0)

* This means that Cutting and Packaging is utilized 100%.

Sowing is not binding (slack > 0)

* This means that we have a surplus of Sowing manpower, which we could potential sell (or use) to increase profit.




#### Impact of shadow prices

When consider the shadow prices we have two options:

* Selling 1 unit of Cutting manpower hours
* Buing 1 unit of Cutting manpower hours

If sell 1 unit we will decrease our profit with 300 while buying 1 unit will increase profit with 300.

This means that if we choose to buy more hours we stop pay no more than 300 while if we sell the hours we should charge at least 300.

Note: selling og buy more sowing hours will not affect the profit

**Decision - Impact of increasing Cutting manpower with 1 unit**

In [9]:
model = LpProblem(name="resournce-allocation", sense=LpMaximize) # Maximise profit

# Initialize the decision variables
x1 = LpVariable(name="x1", lowBound=0, upBound=None, cat='Integer') # x1 => 0
x2 = LpVariable(name="x2", lowBound=0, upBound=None, cat='Integer')

# Add the constraints to the model
model += (0.15 * x1 + 0.1  * x2 <= 111, "cutting (manpower)") # <---- Increasing with 1 unit
model += ( 0.6 * x1 + 0.9  * x2 <= 650, "sowing (manpower)")
model += (0.05 * x1 + 0.05 * x2 <= 40,  "packing (manpower)")


# Add the objective function to the model
model += 80 * x1 + 65 * x2

print(model)

# Solve the problem
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}\n")

# Maximum profit
print(f"objective: {model.objective.value()}\n")

# Optimal values
for var in model.variables():
  print(f"{var.name}: {var.value()}")

# Display Contraints - Shahow price and Slack
o = [{
    'name':name,
    'shadow price':c.pi,
    'slack': c.slack} for name, c in model.constraints.items()]

print(pd.DataFrame(o))

resournce-allocation:
MAXIMIZE
80*x1 + 65*x2 + 0
SUBJECT TO
cutting_(manpower): 0.15 x1 + 0.1 x2 <= 111

sowing_(manpower): 0.6 x1 + 0.9 x2 <= 650

packing_(manpower): 0.05 x1 + 0.05 x2 <= 40

VARIABLES
0 <= x1 Integer
0 <= x2 Integer

status: 1, Optimal

objective: 61300.0

x1: 620.0
x2: 180.0
                 name  shadow price  slack
0  cutting_(manpower)          -0.0   -0.0
1   sowing_(manpower)          -0.0  116.0
2  packing_(manpower)          -0.0   -0.0


As seen in the above output, buying 1 unit of Cutting manpower increase the profit with 300.

However the decision to buy more manpower also affects production volume. We now product more of Product 1 and less of Product 2. Moreover, we also see a, increase in the surpluse of Sowing manpower.

#### Impact of optimizing manpower

Another interesting question could be what would happen if can optimize the manpower across departments. Forexample, let us say we are able to optimize the production layout so we are able to move some manpower around while keeping the same amount of manpower (110 + 650 + 40 = 800)

We model this by introducing two new variables and changing the last constraint.

In [8]:
# Create the model
model = LpProblem(name="Test", sense=LpMaximize)

# Initialize the decision variables
x1 = LpVariable(name="x1", lowBound=0, upBound=None, cat='Integer') # x1 => 0
x2 = LpVariable(name="x2", lowBound=0, upBound=None, cat='Integer')

p = LpVariable(name="p", lowBound=0, upBound=None, cat='Continuous')
q = LpVariable(name="q", lowBound=0, upBound=None, cat='Continuous')

model += (0.15 * x1 + 0.1  * x2 <= p, "cutting (manpower)")
model += ( 0.6 * x1 + 0.9  * x2 <= q, "sowing (manpower)")
model += (0.05 * x1 + 0.05 * x2 <= 800 - p - q,  "packing (manpower)")

# Add the objective function to the model
model += 80*x1 + 65*x2

# Solve the problem
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
  print(f"{var.name}: {var.value()}")


status: 1, Optimal
objective: 80000.0
p: 150.0
q: 600.0
x1: 1000.0
x2: 0.0


**Observation**
When optimizing the manpower mix we increase the profit significantly.

The allocation of manpower also changes:

* Cutting: 150 (previously 110)
* Sowing: 600  (previously 650)
* Packing: 50 (Previously 40)

The product output also changes:

* Now we produce 1000 units of Product 1 and 0 units of Product 2. Interestingly, the total output is also higher, as it was 800 units in the original problem.

This raises new and interesting business questions:
* Can we actually sell 1000 units of Product 1?
* Will we loose customers if dont sell Product 2? (this might happen if there are some syngergies between the products)
* Will losing customers affect the Product 1 sales?
* Do we have enough warehouse capacity to store 1000 units?
* ...


# Resources

* https://coin-or.github.io/pulp/technical/pulp.html
* https://realpython.com/linear-programming-python/#using-pulp
* https://science-gym.dk/mat/20002010/lin-prog.pdf